In [2]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/TimeSeriesAnomaly/data/modify

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1j1N0u5t0l99N_wfSd5UZvnhugzn5g_NC/TimeSeriesAnomaly/data/modify


In [3]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 14.8 MB/s 
     |████████████████████████████████| 122 kB 62.4 MB/s 
     |████████████████████████████████| 181 kB 62.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 157 kB 73.1 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 157 kB 61.6 MB/s 
     |████████████████████████████████| 156 kB 57.4 MB/s 


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import matplotlib.pyplot as plt
import easydict
from sklearn.preprocessing import StandardScaler
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import time
import math

## Data 

In [7]:
#---# LOAD npy file #---#
Fu_20_normal = np.load('Fu_20_normal.npy')
Fu_21_normal = np.load('Fu_21_normal.npy')
Fu_21_abnormal = np.load('Fu_21_abnormal.npy')
Fu_22_normal = np.load('Fu_22_normal.npy')
Fu_22_abnormal = np.load('Fu_22_abnormal.npy')

Fu_20_normal_10 = np.load('Fu_20_normal_10.npy')
Fu_21_normal_10 = np.load('Fu_21_normal_10.npy')
Fu_21_abnormal_10 = np.load('Fu_21_abnormal_10.npy')
Fu_22_normal_10 = np.load('Fu_22_normal_10.npy')
Fu_22_abnormal_10 = np.load('Fu_22_abnormal_10.npy')

# import sys
# np.set_printoptions(threshold=sys.maxsize) # print all

#---# 확인용 #---#
# plt.figure(figsize=(30,5))
# plt.plot(Fu_22_abnormal_10)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class MyDataset(Dataset):
  def __init__(self, data):
    self.data = np.array(self.sliding_window(data, wandb.config.window_size, wandb.config.stride))
    self.data = self.data.reshape(-1, wandb.config.window_size)
    
  def __getitem__(self, index):
    self.x = self.data[index]
    return (index, torch.Tensor(self.x))
      
  def __len__(self):
    return len(self.data)

  #---# Window #---#
  def sliding_window(self, arr, window_size, stride):
    start_pt = 0
    total_data = []
    while(True) :
      if len(arr) < (start_pt + window_size) : break
      data = arr[start_pt:start_pt+window_size]
      start_pt += stride
      total_data.append(data)
    return total_data

class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.Encoder = nn.Sequential(
      nn.Linear(config.window_size,200),
      nn.BatchNorm1d(200),
      nn.LeakyReLU(),
      nn.Linear(200,100),
      nn.BatchNorm1d(100),
      # nn.LeakyReLU(),
      # nn.Linear(100,50),
      # nn.BatchNorm1d(50),

      # nn.LeakyReLU(),
      # nn.Linear(100,50),
      # nn.BatchNorm1d(50),
      nn.LeakyReLU()
    )
    self.Decoder = nn.Sequential(
      # nn.Linear(50,100),
      # nn.BatchNorm1d(100),
      # nn.LeakyReLU(),
      nn.Linear(100,200),
      nn.BatchNorm1d(200),
      nn.LeakyReLU(),
      # nn.Linear(100,200),
      # nn.BatchNorm1d(200),
      # nn.LeakyReLU(),
      # nn.Linear(200,400),
      # nn.BatchNorm1d(400),
      # nn.LeakyReLU(),
      nn.Linear(200,config.window_size),
    )

    #####
    # self.Encoder = nn.Sequential(
    #   nn.Linear(config.window_size, 2*config.window_size),
    #   nn.BatchNorm1d(2*config.window_size),
    #   nn.LeakyReLU(),
    #   nn.Linear(2*config.window_size, 4*config.window_size),
    #   nn.BatchNorm1d(4*config.window_size),
    #   nn.LeakyReLU(),
    #   # nn.Linear(4*config.window_size, 8*config.window_size),
    #   # nn.BatchNorm1d(8*config.window_size),
    #   # nn.LeakyReLU()
    # )
    # self.Decoder = nn.Sequential(
    #   # nn.Linear(8*config.window_size, 4*config.window_size),
    #   # nn.BatchNorm1d(4*config.window_size),
    #   # nn.LeakyReLU(),
    #   nn.Linear(4*config.window_size, 2*config.window_size),
    #   nn.BatchNorm1d(2*config.window_size),
    #   nn.LeakyReLU(),
    #   nn.Linear(2*config.window_size, config.window_size),
    # )
    
  def forward(self, x):
    x = self.Encoder(x)
    x = self.Decoder(x)
    return x

class Trainer():
  def __init__(self, model, optimizer, train_loader, scheduler, device):
    self.model = model
    self.optimizer = optimizer
    self.train_loader = train_loader
    self.scheduler = scheduler
    self.device = device
    # Loss Function
    # self.criterion = nn.L1Loss().to(self.device) ## L1 loss
    self.criterion = nn.CrossEntropyLoss().to(self.device) ## L2 loss
      
  def fit(self) :
    self.model.to(self.device)
    best_score = 0
    for epoch in range(wandb.config.num_epochs):
      self.model.train()
      train_loss = []
      for idx, x in iter(self.train_loader):
        x = x.float().to(self.device)
        self.optimizer.zero_grad()
        
        _x = self.model(x)
        loss = self.criterion(x, _x)
        
        loss.backward()
        self.optimizer.step()

        train_loss.append(loss.item())

      wandb.log({
              "loss": loss
          })

      ####
      # score = self.validation(self.model, 0.95)
      # print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

      # if self.scheduler is not None: self.scheduler.step(score)

      # if best_score < score:
      #   best_score = score
      #   torch.save(model.module.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
  
  def test(self, test_loader, thr) :
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    pred = []; true = []
    xs = []; _xs = []
    predictions = []
    total_num = math.floor((test.shape[0] - config.window_size) / config.stride)

    self.model.eval()
    with torch.no_grad():
      for idx, x in iter(test_loader):
        x = x.float().to(self.device) # x shape : (batch_size, window_size)
        _x = self.model(x) # prediction
        
        for i in range(x.shape[0]) : # batch_size만큼 돌면서 stride씩만 저장
          if (idx[-1] == total_num) and (i == x.shape[0] - 1) : # 마지막 데이터는 전체 다 넣어주어야 함.
            xs.extend(x[i].numpy().tolist())
            _xs.extend(_x[i].numpy().tolist())
          else :
            xs.extend(x[i][0:config.stride].numpy().tolist())
            _xs.extend(_x[i][0:config.stride].numpy().tolist())


        diff = cos(x, _x).cpu().tolist()
        prediction = np.where(np.array(diff) < thr, 1, 0).tolist() # 1, 0이 들어감
        predictions.extend(prediction)

    return predictions, xs, _xs
    # return f1_score(true, pred, average='macro')

#####################
#---# functions #---#
#####################
def drawing(pred, x, _x) :
  plt.figure(figsize=(30,5))
  plt.plot(x, markersize=1)
  plt.plot(pred, marker='.', markersize=2, color='r', linestyle='None')

  #---# 실제 anomaly 값 구간 #---#
  a = np.linspace(62200, 65300)
  plt.fill_between(a, 0, 2000, color='green', alpha=0.3)
  # plt.fill_between(a, -1, 4, color='green', alpha=0.5)
  b = np.linspace(95600, 99300)
  plt.fill_between(b, 0, 2000, color='green', alpha=0.3)
  # plt.fill_between(b, -1, 4, color='green', alpha=0.5)
  c = np.linspace(148400, 152300)
  plt.fill_between(c, 0, 2000, color='green', alpha=0.3)
  # plt.fill_between(c, -1, 4, color='green', alpha=0.5)

  plt.show()
  plt.clf()

def get_anomaly_time(original, prediction) : 
  temp = pd.DataFrame(index=range(0, len(original)), columns={'Fu'})
  # temp = temp.fillna(0)

  for i in range(len(prediction)) :
    if prediction[i] == 0 :
      for j in range(i*wandb.config.stride, (i*wandb.config.stride + wandb.config.window_size)) : 
        temp.loc[j] = np.nan

    elif prediction[i] == 1 : # anomaly
      for j in range(i*wandb.config.stride, (i*wandb.config.stride + wandb.config.window_size)) : 
        try : temp.loc[j] = original[j]
        except : pass

  # anomaly = temp.dropna()
  return temp

def calculate(true_list, pred_list) : ### 이따 수정 예정
  pred_list = pred_list.dropna()
  pred_anomaly_set = set(pred_list.index.tolist())
  pred_normal_set = set(range(len(true_list))) - pred_anomaly_set
  true_anomaly_set = set(np.where(np.array(true_list) == 1)[0].tolist())
  true_normal_set = set(np.where(np.array(true_list) == 0)[0].tolist())
  recall = len(pred_anomaly_set.intersection(true_anomaly_set)) / len(true_anomaly_set)
  # accuracy = len(anomaly_set.intersection(true_set)) / len(anomaly_set.union(true_set))

  accuracy = (len(pred_anomaly_set.intersection(true_anomaly_set)) + len(pred_normal_set.intersection(true_normal_set))) / len(true_list) # (빨간 거 맞은거 + 파란거 맞은거) / 전체
  return recall, accuracy

In [12]:
def main() :
  ##########################
  #---# true list 만들기 #---#
  ##########################
  true_label = [0 for i in range(len(x))] # test_scale
  true_label[62200:65300] = [1 for i in range(62200,65300)]
  true_label[95600:99300] = [1 for i in range(95600,99300)]
  true_label[148400:152300] = [1 for i in range(148400,152300)]

  tm = time.localtime(time.time())
  string = time.strftime('%Y%m%d_%H%M%S', tm)

  # wandb.init(project="Anomaly-Oil", entity="sohyun", name=string, magic=True)

  train = pd.DataFrame(Fu_22_normal_10, columns=['Fu'])
  train = pd.concat([train, pd.DataFrame(Fu_21_normal_10, columns=['Fu']), pd.DataFrame(Fu_20_normal_10, columns=['Fu'])], axis=0)
  test = pd.DataFrame(Fu_22_abnormal_10, columns=['Fu'])
  test_len = len(test)
  total = pd.DataFrame(pd.concat([train, test], axis=0))

  train_scale = train['Fu'].values
  test_scale = test['Fu'].values
  total_scale = total['Fu'].values

  #---# Noramlize #---#
  # scaler = StandardScaler()
  # total_scale = scaler.fit_transform(total); total_scale = pd.DataFrame(total_scale, columns=['Fu'])['Fu'].values.tolist() # total 먼저 해놓고 transform
  # train_scale = scaler.transform(train); train_scale = pd.DataFrame(train_scale, columns=['Fu'])['Fu'].values.tolist()
  # test_scale = scaler.transform(test); test_scale = pd.DataFrame(test_scale, columns=['Fu'])['Fu'].values.tolist()

  #---# Setting train data #---#
  train_dataset = MyDataset(data=train_scale)
  train_loader = DataLoader(train_dataset, batch_size=wandb.config.batch_size, shuffle=True, drop_last=True)
  test_dataset = MyDataset(data=test_scale)
  test_loader = DataLoader(test_dataset, batch_size=wandb.config.batch_size, shuffle=False)

  # model = nn.DataParallel(AutoEncoder())
  model = AutoEncoder()

  optimizer = torch.optim.Adam(params = model.parameters(), lr = config.lr) # lr = config.lr
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

  trainer = Trainer(model, optimizer, train_loader, scheduler, device)
  trainer.fit()
  pred, x, _x = trainer.test(test_loader, wandb.config.threshold)

  a = get_anomaly_time(x, pred)
  recall, accuracy = calculate(true_label, a)

  drawing(a, x, _x) # draw plot
  print(f"===== RECALL ======\n{recall}\n====== ACCURACY =====\n{accuracy}")



In [10]:
# config = easydict.EasyDict({
#     "num_epochs" : 200, #500
#     "batch_size" : 16, #16 
#     "mode" : 'train',
#     # "mode" : "test",
#     "lr" : 1e-3, 
#     "wd" : None,
#     "window_size" : 1000,
#     "stride" : 500,
#     "threshold" : 0.3 # 0.3이나 0.2로 하기
# })

config = {
    'method' : 'grid',
    'parameters' : {
        'lr' :{'values' : [1e-3]}, #1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1
        'batch_size' :{'values' : [8]},
        'window_size' : {'values' : [800, 1000]},
        'stride' : {'values' : [200, 250]},
        'threshold' :{'values' : [0.1, 0.3, 0.5, 0.7, 0.9]}
    }
}
tm = time.localtime(time.time())
string = time.strftime('%Y%m%d_%H%M%S', tm)

wandb.init(project="Anomaly-Oil", entity="sohyun", name=string, magic=True)
sweep_id = wandb.sweep(config, project="Anomaly_oil")
wandb.agent(sweep_id, function=main)

wandb: Currently logged in as: sohyun. Use `wandb login --relogin` to force relogin


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 1040, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 247, in setup
    magic_install(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/integration/magic.py", line 542, in magic_install
    _monkey_tfkeras()
  File "/usr/local/lib/python3.7/dist-packages/wandb/integration/magic.py", line 339, in _monkey_tfkeras
    from wandb.integration.keras import WandbCallback  # add keras import hooks first
  File "/usr/local/lib/python3.7/dist-packages/wandb/integration/keras/__init__.py", line 7, in <module>
    from .keras import WandbCallback
  File "/usr/local/lib/python3.7/dist-packages/wandb/integration/keras/keras.py", line 20, in <module>
    import tensorflow.keras.backend as K
ModuleNotFoundError: No module named 'tensorflow.keras'
wandb: ERROR Abnormal program exit


Exception: ignored